## Task 4.8 - Grouping Data & Aggregating Variables

### 01. Import Libraries and data path

In [1]:
#import libraries
import pandas as pd 
import numpy as np
import os

In [2]:
path = r'C:\Users\dmksk\Desktop\CF\Data Immersion\Achivment 4\4.8\Instacart_Project'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_derived.pkl'))

### 02. aggregated mean of the “order_number” column grouped by “department_id”

In [4]:
#Checking dataframe 
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,Mid-range product,Regularly busy,Regularly busy,Most orders
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,Mid-range product,Regularly busy,Regularly busy,Average orders
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,Mid-range product,Busiest day,Busiest days,Average orders
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0,both,Mid-range product,Regularly busy,Slowest days,Most orders
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,Mid-range product,Least busy,Slowest days,Average orders


In [5]:
# perform single aggregation mean of order number by department_id (average number of orders per user department id)
ords_prods_merge.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### 03. There are 21 department_ids

### 04. Creating flags for Loyalty customers

In [6]:
# Creating new column "max_order"
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
# Checking for new column
ords_prods_merge.head(5)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,Mid-range product,Regularly busy,Regularly busy,Most orders,32
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,Mid-range product,Regularly busy,Regularly busy,Average orders,32
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,Mid-range product,Busiest day,Busiest days,Average orders,5
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0,both,Mid-range product,Regularly busy,Slowest days,Most orders,3
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,Mid-range product,Least busy,Slowest days,Average orders,3


In [8]:
# Assign loyalty flag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [9]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [10]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
# Check loyalty flags
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)


Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [12]:
# Check Loyalty flag in dataframe
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(20)

,user_id,loyalty_flag,order_number
0,138,Regular customer,28
1,138,Regular customer,30
2,709,New customer,2
3,764,New customer,1
4,764,New customer,3
5,777,Regular customer,16
6,825,New customer,3
7,910,Regular customer,12
8,1052,Regular customer,10
9,1052,Regular customer,15


### 05. Customers spending habits comparison¶

In [13]:
# the min max and mean of each loyalty flag
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

### 06. Create a spending flag for each user based on the average price

In [14]:
# Creating new column "Average_spending"
ords_prods_merge['average_spending'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [15]:
# Checking for new column
ords_prods_merge.head(5)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,5,0,both,Mid-range product,Regularly busy,Regularly busy,Most orders,32,Regular customer,6.935811
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,1,both,Mid-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer,6.935811
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,20,0,both,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,7.930208
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,10,0,both,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer,4.972414
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,11,1,both,Mid-range product,Least busy,Slowest days,Average orders,3,New customer,4.972414


In [16]:
# Assign Spending flag for low spenders
ords_prods_merge.loc[ords_prods_merge['average_spending'] < 10.0, 'spending_flag'] = 'Low spender'

In [17]:
# Assign Spending flag for high spenders
ords_prods_merge.loc[ords_prods_merge['average_spending'] >= 10.0, 'spending_flag'] = 'High spender'

In [18]:
# Check spender flags
ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low spender     31770746
High spender      634113
Name: spending_flag, dtype: int64

In [19]:
ords_prods_merge[['user_id', 'average_spending', 'spending_flag', 'prices']].head(10)

,user_id,average_spending,spending_flag,prices
0,138,6.935811,Low spender,5.8
1,138,6.935811,Low spender,5.8
2,709,7.930208,Low spender,5.8
3,764,4.972414,Low spender,5.8
4,764,4.972414,Low spender,5.8
5,777,6.935398,Low spender,5.8
6,825,5.957576,Low spender,5.8
7,910,6.680000,Low spender,5.8
8,1052,7.162500,Low spender,5.8
9,1052,7.162500,Low spender,5.8


### 07. Create a flag for frequent users

In [20]:
# Create a column for frequet users
ords_prods_merge['Customer_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [21]:
# Cheaking for new column
ords_prods_merge.head(10)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending,spending_flag,Customer_frequency
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,both,Mid-range product,Regularly busy,Regularly busy,Most orders,32,Regular customer,6.935811,Low spender,8.0
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer,6.935811,Low spender,8.0
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,both,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,7.930208,Low spender,6.0
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,both,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer,4.972414,Low spender,9.0
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,both,Mid-range product,Least busy,Slowest days,Average orders,3,New customer,4.972414,Low spender,9.0
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,...,both,Mid-range product,Regularly busy,Busiest days,Average orders,26,Regular customer,6.935398,Low spender,11.0
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,...,both,Mid-range product,Regularly busy,Regularly busy,Most orders,9,New customer,5.957576,Low spender,19.0
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,...,both,Mid-range product,Regularly busy,Slowest days,Most orders,12,Regular customer,6.680000,Low spender,3.5
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,...,both,Mid-range product,Regularly busy,Busiest days,Average orders,20,Regular customer,7.162500,Low spender,9.5
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,...,both,Mid-range product,Regularly busy,Busiest days,Most orders,20,Regular customer,7.162500,Low spender,9.5


In [22]:
# Creating flag on order frequency
ords_prods_merge.loc[ords_prods_merge['Customer_frequency'] > 20, 'frequency_flag'] = 'Non_frequent_customer'

In [23]:
ords_prods_merge.loc[(ords_prods_merge['Customer_frequency']>10) & (ords_prods_merge['Customer_frequency']<=20),
                     'frequency_flag'] = 'Regular customer'

In [24]:
ords_prods_merge.loc[ords_prods_merge['Customer_frequency'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [25]:
# Check frequency flags
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent Customer        22796659
Regular customer          6921472
Non_frequent_customer     2686728
Name: frequency_flag, dtype: int64

In [26]:
#Checking for columns
ords_prods_merge[['user_id', 'Customer_frequency', 'frequency_flag']].head(10)

,user_id,Customer_frequency,frequency_flag
0,138,8.0,Frequent Customer
1,138,8.0,Frequent Customer
2,709,6.0,Frequent Customer
3,764,9.0,Frequent Customer
4,764,9.0,Frequent Customer
5,777,11.0,Regular customer
6,825,19.0,Regular customer
7,910,3.5,Frequent Customer
8,1052,9.5,Frequent Customer
9,1052,9.5,Frequent Customer


### Export data as pkl

In [27]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'aggregated_ords_prods_merge.pkl'))